In [ ]:
import random
import json
from faker import Faker
from datetime import datetime
from azure.eventhub import EventHubProducerClient, EventData
from azure.identity import ClientSecretCredential
import time
import pandas as pd
import threading

# SPN e o nome do Event Hub
azure_tenant_id: str = "dc25df03-ffa5-4111-b188-46fe6cd26a3a" #os.getenv('AZURE_TENANT_ID')
azure_client_id: str = "7060731f-a210-43a3-8217-e6741e224f03" #os.getenv('AZURE_CLIENT_ID')
azure_client_secret: str = "" #os.getenv('AZURE_CLIENT_SECRET')
eventhub_namespace: str = "fraud-eh-namespace" #os.getenv('EVENTHUB_NAMESPACE')
eventhub_namespace_fully: str = f"{eventhub_namespace}.servicebus.windows.net"
eventhub_name: str = "fraud-detect" #os.getenv('EVENTHUB_NAME')


# Caminho para o arquivo CSV
file_path = 'phonedataset.csv'

# Lê o arquivo CSV
df = pd.read_csv(file_path, sep=';')
phone_list = df.values.tolist()

# Inicializa o gerador de dados fictícios
fake = Faker()

# Definindo intervalo de tempo

start_date = datetime(2024, 8, 1, 0)
end_date = datetime(2024, 8, 1, 23)

def spn_authentication():
    
    print("init procces")
    credential = ClientSecretCredential(tenant_id=azure_tenant_id, 
                                        client_id=azure_client_id,
                                        client_secret=azure_client_secret)
    print("credential create")
    producer = EventHubProducerClient(fully_qualified_namespace=eventhub_namespace_fully,
                                        eventhub_name=eventhub_name,
                                        credential=credential)
    print("producer create")

    return producer

def generate_random_data():
    """Gera um dicionário com dados aleatórios no formato JSON especificado."""

    brand_version = random.choice(phone_list)
    phone_brand = brand_version[0]
    operational_version = brand_version[1]

    return {
        "imei": str(fake.random_number(digits=15, fix_len=True)),
        "mac": ':'.join([f"{random.randint(0, 255):02X}" for _ in range(6)]),
        "network": random.choice(["VIVO", "TIM", "CLARO", "OI"]),
        "client_ip": fake.ipv4(),
        "latitude": f"{random.uniform(-33.7, 5.3):.6f}",  # Limita a latitude ao Brasil
        "longitude": f"{random.uniform(-73.9, -34.8):.6f}",  # Limita a longitude ao Brasil
        "uid": str(fake.random_number(digits=11, fix_len=True)),
        "password": fake.password(length=6, special_chars=False, digits=True, upper_case=True, lower_case=True),
        "transaction": random.choice(["true", "false"]),
        "api": "login-authentication",
        "endpoint": "v1/login",
        "os": operational_version,
        "phone_brand": phone_brand,
        "app_version": random.choice(["2.0.1", "2.0.2", "2.0.3"]),
        "error": random.choice(["0", "INCORRECT_PASS", "USER_NOT_FOUND"]),
        "timestamp": str(fake.unix_time(start_datetime=start_date, end_datetime=end_date)*1000)
    }


def send_message_to_eventhub(eh_producer, msg_number, trh_number):
    """Abre client com eventhub e envia batch de mensagens."""
    for i in range(msg_number):

        message = json.dumps(generate_random_data())

        try:
            with eh_producer:
                event_data_batch = eh_producer.create_batch()

                event_data_batch.add(EventData(message))

                eh_producer.send_batch(event_data_batch)
                
                print(f"[{trh_number}] - Mensagem enviada com sucesso! - {message}")
        except ValueError as ve:
            print(f"Erro ao enviar mensagem: {ve}")
        except Exception as e:
            print(f"Erro inesperado ao enviar mensagem: {e}")
        finally:
            eh_producer.close()

eh_producer = spn_authentication()

msg_number = 200

thread_1 = threading.Thread(target=send_message_to_eventhub, args = (eh_producer, msg_number, 1))
thread_2 = threading.Thread(target=send_message_to_eventhub, args = (eh_producer, msg_number, 2))

thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()


init procces
credential create
producer create
[1] - Mensagem enviada com sucesso! - {"imei": "915964348345384", "mac": "1F:9B:63:36:15:A0", "network": "CLARO", "client_ip": "107.221.14.60", "latitude": "-6.447109", "longitude": "-41.710587", "uid": "72079726246", "password": "4W2Phb", "transaction": "false", "api": "login-authentication", "endpoint": "v1/login", "os": "Android 13 (One UI 5.1)", "phone_brand": "Samsung Galaxy Z Flip 5", "app_version": "2.0.3", "error": "0", "timestamp": "1722488428000.0"}
[2] - Mensagem enviada com sucesso! - {"imei": "904574456470514", "mac": "6D:9F:07:7F:99:82", "network": "OI", "client_ip": "36.115.110.71", "latitude": "-31.523157", "longitude": "-66.532840", "uid": "92558994422", "password": "7VUuwV", "transaction": "false", "api": "login-authentication", "endpoint": "v1/login", "os": "iOS 17", "phone_brand": "iPhone 14 Pro Max", "app_version": "2.0.2", "error": "INCORRECT_PASS", "timestamp": "1722527883000.0"}
[2] - Mensagem enviada com sucesso! -